In [2]:
from seasonstats import *
import statsmodels.api as sm
import pickle

# Step 3: Get End of Season Data

In [7]:
#OFFSEASON DATA
talent = pd.DataFrame()
for year in [2018,2019,2020,2021,2022,2023]:
    talent = pd.concat([talent,getData('talent',{'year':year})])


In [8]:
returning= pd.DataFrame()
for year in [2018,2019,2020,2021,2022,2023]:
    returning = pd.concat([returning,getData('player/returning',{'year':year})])

In [242]:
#Take end of year data (DONE)
#Update the data for offseason data (returning prod & talent)
#Use that dataset as input


#END OF SEASON DATA
offenseend={}
defenseend={}
for year in [2022]:
    with open(f'data/endofseasonratings/{year}offense.pkl','rb') as f:
        offenseend[year] = pickle.load(f)
    with open(f'data/endofseasonratings/{year}defense.pkl','rb') as f:
        defenseend[year] = pickle.load(f)


In [243]:
offseasondata = pd.merge(talent,returning[['season','team','percentPPA']],how='left',left_on=['year','school'],right_on=['season','team']).dropna()
offseasondata['talent']=offseasondata['talent'].astype(float)


In [244]:
#offenseend[2018]
#offseasondata[offseasondata['year']==2018]


prelog = pd.read_pickle("models/preseason.pickle")

offseasondata['expWin']=prelog.predict(sm.add_constant(offseasondata[['percentPPA','talent']]))

# Step 4: Calculate Preseason Ratings for the Next Year

In [245]:
#offenseend

In [246]:
offensepre = {}
defensepre = {}
year=2022 #previous year (year from offenseend)

offensepre[year+1]={}
for metric in offenseend[year].keys():
    offensepre[year+1][metric]={}
    for item in offenseend[year][metric].items():
        item[0] #team name
        newvalue1 = item[1]*.6+1500*.4
        try:
            adjustment = (offseasondata[(offseasondata['year']==(year+1))&(offseasondata['school']==item[0])]['expWin'].iloc[0]-.5)*400
        except: adjustment = 0
        #print(item[0],item[1],int(newvalue1+adjustment), (adjustment))
        offensepre[year+1][metric][item[0]]=int(newvalue1+adjustment)

defensepre[year+1]={}
for metric in defenseend[year].keys():
    defensepre[year+1][metric]={}
    for item in defenseend[year][metric].items():
        item[0] #team name
        newvalue1 = item[1]*.6+1500*.4
        try:
            adjustment = (offseasondata[(offseasondata['year']==(year+1))&(offseasondata['school']==item[0])]['expWin'].iloc[0]-.5)*400
        except: adjustment = 0
        #print(item[0],item[1],int(newvalue1+adjustment), (adjustment))
        defensepre[year+1][metric][item[0]]=int(newvalue1+adjustment)
    



In [247]:


#offensepre[2019]['ypp']['LSU'],offensepre[2019]['TDRatio']['LSU'],offensepre[2019]['TurnoverRatio']['LSU']


In [248]:
#offenseend[2018]['ypp']['LSU'],offenseend[2018]['TDRatio']['LSU'],offenseend[2018]['TurnoverRatio']['LSU']

In [249]:
#offseasondata[(offseasondata['year']==2019)].sort_values("expWin",ascending=False).head(20)

checko = pd.merge(pd.DataFrame.from_dict(offensepre[2019]['TDRatio'],orient='index',columns=['ypp']).sort_values('ypp',ascending=False),
pd.DataFrame.from_dict(offenseend[2018]['TDRatio'],orient='index',columns=['ypp']).sort_values('ypp',ascending=False), how='left',left_index=True,right_index=True, suffixes=('_19pre','_18end'))

checkd = pd.merge(pd.DataFrame.from_dict(defensepre[2019]['TDRatio'],orient='index',columns=['ypp']).sort_values('ypp',ascending=False),
pd.DataFrame.from_dict(defenseend[2018]['TDRatio'],orient='index',columns=['ypp']).sort_values('ypp',ascending=False), how='left',left_index=True,right_index=True, suffixes=('_19pre','_18end'))


check = pd.merge(checko,checkd,how='left',left_index=True,right_index=True,suffixes=("_off","_def"))
check['18total']=check['ypp_18end_off']+check['ypp_18end_def']
check['19total']=check['ypp_19pre_off']+check['ypp_19pre_def']

check['18total']=check['18total'].rank(ascending=False)
check['19total']=check['19total'].rank(ascending=False)

check.sort_values(by='19total',ascending=True).head(20)

check.head(40).assign(change = lambda x: x['18total']-x['19total']).sort_values(by='change',ascending=False)

# Step 5: Export the Preseason Data to be used in the next ELO Model

In [250]:
#EXPORT PRESEASON DATA FOR THE NEXT YEAR
#THIS DATE SHOULD BE ONE YEAR AFTER ALL INPUT DATA'S DATE
'''
with open('data/preseasonratings/2023offense.pkl','wb') as f:
    pickle.dump(offensepre,f)
with open('data/preseasonratings/2023defense.pkl','wb') as f:
    pickle.dump(defensepre,f)

In [164]:
# Return to ELOCalc.ipynb and use the preseason data as input data for next ELO